In [ ]:
#Bash script - processing of wastewater sequencing raw_data

#!/bin/bash
#python sanitize_bed.py "${refs}"swift_primersv2.bed 
#python prepare_amplicon_info.py "${refs}"swift_primersv2.bed "${refs}"swift_primersv2_info.tsv "${refs}"swift_primersv2_info.tsv

module load samtools/1.15
module load bwa/0.7.17

module load vcflib/1.0.1
module load snpEff/5.0

batch=$2
refs="/gpfs/gsfs12/users/Irp-jiang/share/covid_data/WWBE/info/"
base="/gpfs/gsfs12/users/Irp-jiang/share/covid_data/WWBE/${batch}/"

#mkdir "${base}analysis_galaxy/"
analysis="${base}analysis_galaxy/"
#mkdir "${analysis}tsvfiles"
#mkdir "${analysis}vcffiles"
#mkdir "${analysis}merged_file"
#mkdir "${analysis}bamfiles"


#aligning
bwa mem -t 32 -v 1 "${refs}"NC_045512.2.fasta "${base}raw_data/${1}"_1.fastq.gz "${base}raw_data/${1}"_2.fastq.gz | samtools view -@ 32 -b -f 1 -F 268 -q 20 -s 1.0 | samtools sort -@ 32 -o "${analysis}bamfiles/${1}".sorted.bam
echo "Sample was initially aligned with BWA"

#trimming primers and base quality
module load ivar/1.3.1
ivar trim -e -m 1 -q 0 -b "${refs}"swift_primersv2.bed -p "${analysis}bamfiles/${1}".trimmed -i "${analysis}bamfiles/${1}".sorted.bam
echo "Amplification primers were removed"

#lofreq realign - may need to separate samtools to controls the version
module load lofreq/2.1.5
lofreq viterbi --defqual 2 -f "${refs}"NC_045512.2.fasta "${analysis}bamfiles/${1}".trimmed.bam | samtools sort -@ 32 -o "${analysis}bamfiles/${1}".trimmed.realigned.sorted.bam
echo "Bam file was re-aligned using lofreq"

#lofreq indelquality and indexing
lofreq indelqual --dindel -f "${refs}"NC_045512.2.fasta -o "${analysis}bamfiles/${1}".trimmed.realigned.indelqual.bam "${analysis}bamfiles/${1}".trimmed.realigned.sorted.bam 
echo "Bam file annotated with indel information"

#lofreq variant call #1
lofreq call --min-cov 5 --max-depth 1000000 -q 30 -Q 30 -e --min-mq 20 --sig 0.0005 --bonf dynamic --no-default-filter --no-default-filter -f "${refs}"NC_045512.2.fasta --call-indels -o "${analysis}vcffiles/${1}"_trimmed.vcf "${analysis}bamfiles/${1}".trimmed.realigned.indelqual.bam
echo "First round of variant call with lofreq finished"

#lofreq call --min-cov 20  -q 20 -Q 20 -e --min-mq 20 -f "${refs}"NC_045512.2.fasta --call-indels -o "${analysis}vcffiles/${1}"_trimmed.vcf "${analysis}vcffiles/${1}".trimmed.realigned.indelqual.bam
lofreq filter -V 0 -v 5 -a 0.05 -A 0.95 -i "${analysis}vcffiles/${1}"_trimmed.vcf -o "${analysis}vcffiles/${1}"_trimmed_filtered.vcf
echo "Fisrt round of variant call filtered"

#step to revome primer bias reads
module load ivar/1.3.1
ivar getmasked -i "${analysis}vcffiles/${1}"_trimmed_filtered.vcf -b "${refs}"swift_primersv2.bed -f "${refs}"swift_primersv2_info.tsv -p "${analysis}${1}"_primer_mismatchers_indices
python completemask.py "${analysis}${1}"_primer_mismatchers_indices.txt "${refs}"swift_primersv2_info.tsv "${analysis}${1}"_primer_mismatchers_indices_v2.txt
ivar removereads -i "${analysis}bamfiles/${1}".trimmed.realigned.indelqual.bam -b "${refs}"swift_primersv2.bed -t "${analysis}${1}"_primer_mismatchers_indices_v2.txt -p "${analysis}bamfiles/${1}".trimmed.realigned.indelqual.readsrem.bam
echo "Bias reads removed with ivar"

#lofreq variant call #2
module load lofreq/2.1.5
#lofreq call --min-cov 20  -q 20 -Q 20 -e --min-mq 20 -f "${refs}"NC_045512.2.fasta --call-indels -o "${analysis}vcffiles/${1}"_trimmed_v2.vcf "${analysis}vcffiles/${1}".trimmed.realigned.indelqual.readsrem.bam
lofreq call --min-cov 5 --max-depth 1000000 -q 30 -Q 30 -e --min-mq 20 --sig 0.0005 --bonf dynamic --no-default-filter --no-default-filter -f "${refs}"NC_045512.2.fasta --call-indels -o "${analysis}vcffiles/${1}"_trimmed_v2.vcf "${analysis}bamfiles/${1}".trimmed.realigned.indelqual.readsrem.bam
echo "Second round of variant call with lofreq finished - using removed reads bam"

#vcf files intersect #1
vcfintersect -r "${refs}"NC_045512.2.fasta -v -w 0  -i "${analysis}vcffiles/${1}"_trimmed_v2.vcf "${analysis}vcffiles/${1}"_trimmed_filtered.vcf > "${analysis}vcffiles/${1}"_trimmed_intersect.vcf
echo "VCF_trimmed_v2 intersected with vcf_trimmed_filtered"

#re-naming
sed -r --sandbox -e 's/^(#CHROM.+)$/##FILTER=<ID=AmpliconRemoval,Description="Variant removed upon removal of amplicon">\n\1/g' -e 's/(.+\t)PASS(\t.+)/\1AmpliconRemoval\2/g' "${analysis}vcffiles/${1}"_trimmed_intersect.vcf > "${analysis}vcffiles/${1}"_trimmed_renamed.vcf
echo "Intersected vcf re-named"

#vcf files union
vcfintersect -r "${refs}"NC_045512.2.fasta -w 0 -u "${analysis}vcffiles/${1}"_trimmed_renamed.vcf  "${analysis}vcffiles/${1}"_trimmed_v2.vcf > "${analysis}vcffiles/${1}"_trimmed_union.vcf
lofreq filter -V 0 -v 0 -a 0.0 -A 0.0 -b fdr -c 0.001 --print-all -i "${analysis}vcffiles/${1}"_trimmed_union.vcf -o "${analysis}vcffiles/${1}"_trimmed_union_filtered.vcf
echo "Final vcf created - union of vcf_trimmed_renamed with vcf_trimmed_v2"

#annotation
#wget https://snpeff.blob.core.windows.net/databases/v5_0/snpEff_v5_0_NC_045512.2.zip
#unzip snpEff_v5_0_NC_045512.2.zip will create data folder
snpEff eff -nodownload -dataDir ${refs}data -i vcf -o vcf -formatEff -classic -no-downstream -no-intergenic -no-upstream -ud 0 -stats stats.html -noLog NC_045512.2 "${analysis}vcffiles/${1}"_trimmed_union_filtered.vcf > "${analysis}vcffiles/${1}"_trimmed_union_snpEff.vcf
echo "Final VCF annotated"

python "${refs}"add_variant_tsv_improved.py  "${analysis}vcffiles/${1}"_trimmed_union_snpEff.vcf  "${analysis}vcffiles/${1}"_trimmed_union_snpEff_final.tsv

module load samtools/1.15
samtools fastq -1 "${analysis}fastqfiles/${1}"_1.fq.gz -2 "${analysis}fastqfiles/${1}"_2.fq.gz -@ 32 -n "${analysis}bamfiles/${1}".trimmed.realigned.indelqual.bam
echo "Generated FASTQ files for SRA submission"

After processing of raw data - get metrics of the sequencing batch

In [ ]:
#Get metrics of the sequencing batch

module load samtools
echo -e "File \tMean \tBreadth \tRead_count" >> /data/salgadofontenr2/ww_cov_2021/metrics_galaxy_${1}.txt

for file in /gpfs/gsfs12/users/Irp-jiang/share/covid_data/WWBE/${1}/analysis_galaxy/bamfiles/*.trimmed.realigned.indelqual.readsrem.bam;
do
	prefix=$(echo $file | cut -d "/" -f 12 | cut -d "." -f 1)
	count=$(samtools view -c -F 4 "${file}")
	mean=$(samtools depth -a "${file}" | awk '{c++;s+=$3}END{print s/c}')
	breadth=$(samtools depth -a "${file}" | awk '{c++; if($3>10) total+=1}END{print (total/(c))*100}')
	echo -e "${prefix} \t${mean} \t${breadth} \t${count}" >> /data/salgadofontenr2/ww_cov_2021/metrics_galaxy_${1}.txt
done


Python code to edit VCF file after raw data processing - add_variant_tsv_improved.py

In [ ]:
import pandas as pd
import numpy as np
import sys
import vcf

GENE_MAP = {'ORF1a': [266, 13468],
            'ORF1b': [13468, 21555],
            'S': [21563, 25384],
            'ORF3a': [25393, 26220],
            'E': [26245, 26472],
            'M': [26523, 27191],
            'ORF6': [27202, 27387],
            'ORF7a': [27394, 27759],
            'ORF7b': [27756, 27887],
            'ORF8': [27894, 28259],
            'N': [28274, 29533],
            'ORF10': [29558, 29674]}

def read_lofreq(filename):
    lofreq_calls = pd.DataFrame(columns=["REGION", "POS", "REF", "ALT", "QUAL", "FILTER", 
                                         "REF_DP", "REF_RV", "ALT_DP", "ALT_RV",
                                         "AF", "TOTAL_DP", "STRAND-BIAS", "EFFECT"])
    vcf_reader = vcf.Reader(filename=filename)
    for row in vcf_reader:
        if len(row.FILTER) == 0 and "EFF" not in row.INFO:
            lofreq_calls = lofreq_calls.append({"REGION": row.CHROM, 
                                            "POS": int(row.POS),
                                            "REF": str(row.REF),
                                            "ALT": str(row.ALT[0]),
                                            "QUAL": row.QUAL, 
                                            "FILTER": "",
                                            "REF_DP": row.INFO["DP4"][0] + row.INFO["DP4"][1], 
                                            "REF_RV": row.INFO["DP4"][1],
                                            "ALT_DP": row.INFO["DP4"][2] + row.INFO["DP4"][3],
                                            "ALT_RV": row.INFO["DP4"][3],
                                            "AF": row.INFO["AF"],
                                            "TOTAL_DP": row.INFO["DP"],
                                            "STRAND-BIAS":row.INFO["SB"],
                                            "EFF":""
                                           }, 
                                           ignore_index=True)
        if len(row.FILTER) > 0 and "EFF" in row.INFO:
            lofreq_calls = lofreq_calls.append({"REGION": row.CHROM, 
                                            "POS": int(row.POS),
                                            "REF": str(row.REF),
                                            "ALT": str(row.ALT[0]),
                                            "QUAL": row.QUAL, 
                                            "FILTER": row.FILTER[0],
                                            "REF_DP": row.INFO["DP4"][0] + row.INFO["DP4"][1], 
                                            "REF_RV": row.INFO["DP4"][1],
                                            "ALT_DP": row.INFO["DP4"][2] + row.INFO["DP4"][3],
                                            "ALT_RV": row.INFO["DP4"][3],
                                            "AF": row.INFO["AF"],
                                            "TOTAL_DP": row.INFO["DP"],
                                            "STRAND-BIAS":row.INFO["SB"],
                                            "EFF":row.INFO["EFF"]
                                           }, 
                                           ignore_index=True)
        if len(row.FILTER) == 0 and "EFF" in row.INFO:
            lofreq_calls = lofreq_calls.append({"REGION": row.CHROM, 
                                            "POS": int(row.POS),
                                            "REF": str(row.REF),
                                            "ALT": str(row.ALT[0]),
                                            "QUAL": row.QUAL, 
                                            "FILTER": "",
                                            "REF_DP": row.INFO["DP4"][0] + row.INFO["DP4"][1], 
                                            "REF_RV": row.INFO["DP4"][1],
                                            "ALT_DP": row.INFO["DP4"][2] + row.INFO["DP4"][3],
                                            "ALT_RV": row.INFO["DP4"][3],
                                            "AF": row.INFO["AF"],
                                            "TOTAL_DP": row.INFO["DP"],
                                            "STRAND-BIAS":row.INFO["SB"],
                                            "EFF":row.INFO["EFF"],
                                           }, 
                                           ignore_index=True)
        if len(row.FILTER) > 0 and "EFF" not in row.INFO:
            lofreq_calls = lofreq_calls.append({"REGION": row.CHROM, 
                                            "POS": int(row.POS),
                                            "REF": str(row.REF),
                                            "ALT": str(row.ALT[0]),
                                            "QUAL": row.QUAL, 
                                            "FILTER": row.FILTER[0],
                                            "REF_DP": row.INFO["DP4"][0] + row.INFO["DP4"][1], 
                                            "REF_RV": row.INFO["DP4"][1],
                                            "ALT_DP": row.INFO["DP4"][2] + row.INFO["DP4"][3],
                                            "ALT_RV": row.INFO["DP4"][3],
                                            "AF": row.INFO["AF"],
                                            "TOTAL_DP": row.INFO["DP"],
                                            "STRAND-BIAS":row.INFO["SB"],
                                            "EFF":""
                                           }, 
                                           ignore_index=True)
    if lofreq_calls.empty:
        return lofreq_calls
    lofreq_calls["Variant"] = lofreq_calls.apply(lambda row:
                                         str(row["POS"]) + ":" + \
                                         str(row["ALT"][1:]) if len(row["ALT"]) > 1 else \
                                         (str(row["POS"]) + "-" + \
                                         str(row["POS"] + len(row["REF"][1:]) + 1) if len(row["REF"]) > 1 else \
                                         str(row["REF"]) + str(row["POS"]) + \
                                         str(row["ALT"])), axis=1)
    lofreq_calls["EFFECT"] = lofreq_calls.apply(lambda row:
                                         str(row["EFF"]).split("|")[1] if row["EFF"] else \
                                         "SILENT", axis=1)
    lofreq_calls["AA_MUT"] = lofreq_calls.apply(lambda row:
                                         str(row["EFF"]).split("|")[3] if row["EFF"] else \
                                         "", axis=1)
    lofreq_calls["GENE"] = lofreq_calls.apply(lambda row:
                                         str(row["EFF"]).split("|")[5] if row["EFF"] else \
                                         "Non-coding", axis=1)
    #lofreq_calls = lofreq_calls[lofreq_calls["FILTER"] != "AmpliconRemoval"]
    return lofreq_calls

def main():
    file = sys.argv[1]
    new_file = read_lofreq(file)
    new_file.to_csv(sys.argv[2], sep="\t", index=False)

if __name__ == "__main__":
    main()